In [1]:
# Install any additional libraries not pre-installed on Kaggle

!pip install langchain chromadb openai beautifulsoup4 --quiet

# Standard libraries
import os
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Additional libraries from pip
import langchain
import chromadb
import openai

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print("All libraries imported successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.1 MB/s eta 0:00:00


In [2]:
os.makedirs('/kaggle/working/raw', exist_ok=True)
os.makedirs('/kaggle/working/processed', exist_ok=True)
os.makedirs('/kaggle/working/metadata', exist_ok=True)

print("✅ Basic directory structure created successfully!")

✅ Basic directory structure created successfully!


In [3]:
!ls -l /kaggle/working/

total 44
drwxr-xr-x 2 root root  4096 Apr  8 21:12 metadata
---------- 1 root root 28787 Apr  8 21:12 __notebook__.ipynb
drwxr-xr-x 2 root root  4096 Apr  8 21:12 processed
drwxr-xr-x 2 root root  4096 Apr  8 21:12 raw


In [4]:
!ls -l /kaggle/working/raw/

total 0


In [5]:
!cp /kaggle/input/nem-test-pdf/nem_fact_sheet.pdf /kaggle/working/raw/

In [6]:
!ls -l /kaggle/working/raw/

total 144
-rw-r--r-- 1 root root 145957 Apr  8 21:12 nem_fact_sheet.pdf


In [7]:
!pip install PyPDF2 --quiet
from PyPDF2 import PdfReader

pdf_path = "/kaggle/working/raw/nem_fact_sheet.pdf"

reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text()

print("🔍 PDF Content Preview:\n")
print(text)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
🔍 PDF Content Preview:

 
 Fact Sheet:  
Modernizing NEM to Meet California’s Reliability and Climate  Goals  
Proceeding R.20-08-020 | www.cpuc.ca.gov/nemrevisit  | November 10, 2022  
CPUC Releases New Proposed Decision to  
Update the Net Energy Metering (NEM)  Tariff to Support the Modern Grid  
The Proposal Promotes Solar and Battery Storage, Supports Grid Reliability,  and 
Controls Electricity Costs for All Californians  
 
• On November 10, 2022, the California Public Utilities Commission ( CPUC ) released a new 
proposal  to modernize rooftop solar and storage  through updated incentives .  
• New r esidential customer s install ing solar will save $100 a month on their electricity bill s on 
average,  and those installing solar and battery storage will save  at least  $136 a month  on 
average.  With th ese savings, these customers will fully pay off their solar systems in just nine 
years or less